In [1]:
'''
Decision Tree 알고리즘
 
 data set
 --------------------------- 
 먹구름(x1) 돌풍(x2) 비(y) 
    1       1     yes
    1       1     yes
    1       0     no
    0       1     no
    0       1     no
  ---------------------------
  [해설] x1=0 -> y=0, x2=0 -> y=0, x2=1 ->     
'''
from math import log # 자연로그(엔트로피)

# 1. data set 생성 
def createDataSet():
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    columns = ['dark clouds','gust'] # dark clouds, gust, label 
    return dataSet, columns

dataSet, columns = createDataSet()
print(dataSet)
print(columns)


[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
['dark clouds', 'gust']


In [2]:
# 2. 기준 엔트로피 계산
def calcShannonEnt(dataSet):
    numEntries = len(dataSet) # entry=관측치 수 : 5
    
    labelCounts = {} # 빈 set
    for featVec in dataSet: # 행 단위 넘김 
        currentLabel = featVec[-1] # 3번째 원소(label) 선택 
        # yes/no count
        labelCounts[currentLabel] = labelCounts.get(currentLabel, 0)+1
        #print('labelCounts :', labelCounts)    
    '''         
    labelCounts : {'yes': 2, 'no': 3}
    '''
    shannonEnt = 0.0  # 엔트로피 초기화 
    for key in labelCounts:
        # label 확률 = label 빈도수 / 전체 관측치 수 
        prob = float(labelCounts[key])/numEntries # 확률 : yes:2/5, no:3/5
        
        shannonEnt -= prob * log(prob,2) # 엔트로피 계산 누적 : 확률, 자연로그 이용 
        
    return shannonEnt

baseEnt = calcShannonEnt(dataSet)  
print('전체 엔트로피 :', baseEnt) # 0.9709505944546686 : dataset 전체 엔트로피 


전체 엔트로피 : 0.9709505944546686


In [3]:
# 불확실성 증가   -> 엔트로피 증가 
'''
엔트로피(세년엔트로피) = 불확실성 척도 
  - 불확실성 낮음(확률이 높은 경우) => 엔트로피 최소화  
'''

# 3. data set 분할  : 칼럼/값으로 data 분할  -> 현재 축을 제외한 나머지 축의 값 반환 
def splitDataSet(dataSet, axis, value): #(전체data, 열축, 값)  
    reDataSet = []
    for subVec in dataSet: 
        if subVec[axis] == value : # 칼럼 값 == value
            reducedFeatVec = subVec[:axis] # axis=1 : subVec[:1] -> subVec[0]
            reducedFeatVec.extend(subVec[axis+1:]) # subVec[1+1:] -> subVec[2]=subVec[0,2]
            reDataSet.append(reducedFeatVec) # reDataSet[[0,2], ]
    return reDataSet

In [4]:
# 4. 중요도가 높은 변수 선택  : 어떤 변수를 기준으로 label을 분류할 것인가 결정    
def chooseBestVariable(dataSet):
    numFeatures = len(dataSet[0]) - 1  # 2=3-1
    baseEntropy = calcShannonEnt(dataSet) # 0.9709505944546686 : 기준 엔트로피 
    bestInfoGain = 0.0; bestFeature = -1
    for i in range(numFeatures):  # range(2) : 0~1
        featList = [example[i] for example in dataSet] # 0번째/1번째 요소 각각 저장 
        uniqueVals = set(featList)  # 중복 list 제거 
        newEntropy = 0.0
        for value in uniqueVals: # {0, 1} 
            subDataSet = splitDataSet(dataSet, i, value) # data set 분할
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet) # 각 변수/값 엔트로피 계산    
            
        infoGain = baseEntropy - newEntropy # 정보이득 계산(전체엔트로피-속성엔트로피)  
        if (infoGain > bestInfoGain): # 최대값 알고리즘 
            bestInfoGain = infoGain  
            bestFeature = i # 칼럼 수정 : 정보이득이 가장 큰 칼럼 교체   
    return bestFeature 

infoGain = chooseBestVariable(dataSet)
print('정보이득이 높은 속성 : ', infoGain+1,'번째 칼럼')
# 정보이득이 같은 높은 변수 :  0 -> (첫번째 칼럼)
'''
엔트로피 최소화 = 정보이득 최대 
 - 엔트로피가 가장 낮게 분할하는것이 정보이득를 최대화
 - 정보이득 : 전체 엔트로피 - 변수 엔트로피(x1, x2) 
'''


정보이득이 높은 속성 :  1 번째 칼럼


In [5]:
# 5. 의사결정 트리  생성 함수 
def createTree(dataSet, columns): 
    classList = [example[-1] for example in dataSet] 
     
    # 재귀호출 리턴 조건(list 첫번째 원소 갯수 ==전체 원소 갯수) : label 100% 분류 -> ['no', 'no']
    if classList.count(classList[0]) == len(classList): # end 조건
        return classList[0]  # 재귀호출 리턴 값
    
    bestVariable = chooseBestVariable(dataSet) # 중요도 높은 변수(0) 선택 
    bestFeatLabel = columns[bestVariable] 
    myTree = {bestFeatLabel:{}} # { dark clouds: { } } 초기화  
    del(columns[bestVariable]) 
    featValues = [example[bestVariable] for example in dataSet]
    uniqueVals = set(featValues)

    for value in uniqueVals: # {0, 1}
        subcolumns = columns[:] # 중요 변수(dark clouds)의 하위 변수(gust) 
        # createTree(중요도 변수 이용 칼럼단위 data set 분할, 서브칼럼)
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestVariable, value),subcolumns)
    
    return myTree  
    
 
myTree = createTree(dataSet, columns) # (전체data, label)
print('분류트리 결과')
print(myTree)
# {'dark clouds': {0: 'no', 1: {'gust': {0: 'no', 1: 'yes'}}}}
# [해설] 먹구름이 0이면 no, 먹구름이 1이면서 돌풍이 1이면 yes, 돌풍이 0이면 no
# 재귀호출방식에 의해서 처음 리턴값이 마지막에 추가된다.


분류트리 결과
{'dark clouds': {0: 'no', 1: {'gust': {0: 'no', 1: 'yes'}}}}
